In [1]:
from xgboost.sklearn import XGBRegressor
import pandas as pd

In [2]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
import torch
from model.model_load import load_model
from configs.setting import global_setting
from libs.common.project_paths import GetPaths

import pandas as pd
import matplotlib.pyplot as plt
from libs import train_utils

import joblib

In [4]:
import yaml

def load_config(cfg_path):
    return yaml.full_load(open(cfg_path, 'r', encoding='utf-8-sig'))


In [5]:
df = pd.read_excel('res/total_set.xlsx')

In [6]:
config_file = 'cfg.yaml'
config_path = GetPaths().get_configs_folder(config_file)

config = load_config(config_path)

# Dataset

In [7]:
from sklearn.preprocessing import MinMaxScaler
from libs.common.common import scaler_save

In [8]:
train, valid, test = train_utils.split_dataset(df, config)

In [38]:
weather_cols = ['rain', 'wind', 'humidity', 'pressure', 'snow', 'cloud', 'visibility']
weather_col1 = ['rain', 'wind', 'humidity', 'snow', 'cloud', 'visibility']
weather_col2 = ['rain', 'wind', 'pressure', 'snow', 'cloud', 'visibility']
wc1 = [0,1,2,4,5,6]
wc2 = [0,1,3,4,5,6]


mid_target = ['Vertical', 'Horizontal', 'Module', 'Outdoor']

sensor_col1 = ['Vertical', 'Module']
sensor_col2 = ['Horizontal', 'Outdoor']
sc1 = [0, 2]
sc2 = [1, 3]

target_cols = ['total']


In [44]:
train_weather_x = train[weather_cols]
valid_weather_x = valid[weather_cols]
test_weather_x = test[weather_cols]

train_weather_y = train[mid_target]
valid_weather_y = valid[mid_target]
test_weather_y = test[mid_target]

# Set Scaler
s_x = 'ckpt/ml/2stage-1/SCALER_1/x_scaler.pkl'
s_y = 'ckpt/ml/2stage-1/SCALER_1/y_scaler.pkl'

sc_x = joblib.load(s_x)
sc_y = joblib.load(s_y)


tx = sc_x.transform(train_weather_x)
vx = sc_x.transform(valid_weather_x)
test_x = sc_x.transform(test_weather_x)

ty = sc_y.transform(train_weather_y[mid_target].values)
vy = sc_y.transform(valid_weather_y[mid_target].values)

C:\Users\proto\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\proto\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\proto\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [34]:
tx[:, [1,2]].shape

array([[0.        , 0.18867925, 0.5       , ..., 0.        , 0.3       ,
        0.25534402],
       [0.        , 0.1509434 , 0.62222222, ..., 0.        , 0.7       ,
        0.38259853],
       [0.        , 0.02830189, 0.97777778, ..., 0.        , 1.        ,
        0.01519706],
       ...,
       [0.        , 0.05660377, 0.98888889, ..., 0.        , 1.        ,
        0.09151637],
       [0.        , 0.        , 0.94444444, ..., 0.        , 0.7       ,
        0.34402138],
       [0.        , 0.12264151, 0.95555556, ..., 0.        , 1.        ,
        0.13877756]])

In [37]:
for idx, weather_c in enumerate([wc1, wc2]):
    tx_ = tx[:, weather_c]
    vx_ = vx[:, weather_c]
#     test_weather_x = test[weather_c]
#     test_weather_y = test[mid_target]


    eval_set = [(vx_, vy)]
    
    reg = XGBRegressor(n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
                       max_depth=5, ## 개별 나무의 최대 깊이
                       gamma = 0, ## gamma
                       importance_type='gain', ## gain, weight, cover, total_gain, total_cover
                       reg_lambda = 1, ## tuning parameter of l2 penalty                   
                       random_state=100
                      ).fit(tx_,ty,
                            eval_set = eval_set)
    
    filename = 'ckpt/ml/ensemble/xgb_ens1_{}.model'.format(idx)
    reg.save_model(filename)
    
    
    

[0]	validation_0-rmse:0.26303
[1]	validation_0-rmse:0.23738
[2]	validation_0-rmse:0.22308
[3]	validation_0-rmse:0.21528
[4]	validation_0-rmse:0.21125
[5]	validation_0-rmse:0.20866
[6]	validation_0-rmse:0.20744
[7]	validation_0-rmse:0.20679
[8]	validation_0-rmse:0.20666
[9]	validation_0-rmse:0.20629
[10]	validation_0-rmse:0.20629
[11]	validation_0-rmse:0.20609
[12]	validation_0-rmse:0.20627
[13]	validation_0-rmse:0.20640
[14]	validation_0-rmse:0.20648
[15]	validation_0-rmse:0.20636
[16]	validation_0-rmse:0.20641
[17]	validation_0-rmse:0.20654
[18]	validation_0-rmse:0.20661
[19]	validation_0-rmse:0.20673
[20]	validation_0-rmse:0.20673
[21]	validation_0-rmse:0.20676
[22]	validation_0-rmse:0.20694
[23]	validation_0-rmse:0.20692
[24]	validation_0-rmse:0.20708
[25]	validation_0-rmse:0.20713
[26]	validation_0-rmse:0.20712
[27]	validation_0-rmse:0.20704
[28]	validation_0-rmse:0.20721
[29]	validation_0-rmse:0.20731
[30]	validation_0-rmse:0.20740
[31]	validation_0-rmse:0.20758
[32]	validation_0-

In [40]:
train_weather_x = train[mid_target]
valid_weather_x = valid[mid_target]
test_weather_x = test[mid_target]

train_weather_y = train[target_cols]
valid_weather_y = valid[target_cols]
test_weather_y = test[target_cols]

# Set Scaler
s_x = 'ckpt/ml/2stage-1/SCALER_2/x_scaler.pkl'
s_y = 'ckpt/ml/2stage-1/SCALER_2/y_scaler.pkl'

sc_x = joblib.load(s_x)
sc_y = joblib.load(s_y)

tx = sc_x.transform(train_weather_x)
vx = sc_x.transform(valid_weather_x)

ty = sc_y.transform(train_weather_y[target_cols].values)
vy = sc_y.transform(valid_weather_y[target_cols].values)
test_y = sc_y.transform(test_weather_y[target_cols].values)

C:\Users\proto\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\proto\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [41]:
for idx, weather_c in enumerate([sc1, sc2]):
    tx_ = tx[:, weather_c]
    vx_ = vx[:, weather_c]
#     test_weather_x = test[weather_c]
#     test_weather_y = test[mid_target]


    eval_set = [(vx_, vy)]
    
    reg = XGBRegressor(n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
                       max_depth=5, ## 개별 나무의 최대 깊이
                       gamma = 0, ## gamma
                       importance_type='gain', ## gain, weight, cover, total_gain, total_cover
                       reg_lambda = 1, ## tuning parameter of l2 penalty                   
                       random_state=100
                      ).fit(tx_,ty,
                            eval_set = eval_set)
    
    filename = 'ckpt/ml/ensemble/xgb_ens2_{}.model'.format(idx)
    reg.save_model(filename)
    
    
    

[0]	validation_0-rmse:0.31555
[1]	validation_0-rmse:0.22241
[2]	validation_0-rmse:0.15790
[3]	validation_0-rmse:0.11361
[4]	validation_0-rmse:0.08368
[5]	validation_0-rmse:0.06442
[6]	validation_0-rmse:0.05281
[7]	validation_0-rmse:0.04581
[8]	validation_0-rmse:0.04209
[9]	validation_0-rmse:0.04013
[10]	validation_0-rmse:0.03918
[11]	validation_0-rmse:0.03879
[12]	validation_0-rmse:0.03857
[13]	validation_0-rmse:0.03849
[14]	validation_0-rmse:0.03847
[15]	validation_0-rmse:0.03846
[16]	validation_0-rmse:0.03853
[17]	validation_0-rmse:0.03853
[18]	validation_0-rmse:0.03855
[19]	validation_0-rmse:0.03856
[20]	validation_0-rmse:0.03855
[21]	validation_0-rmse:0.03855
[22]	validation_0-rmse:0.03856
[23]	validation_0-rmse:0.03856
[24]	validation_0-rmse:0.03856
[25]	validation_0-rmse:0.03858
[26]	validation_0-rmse:0.03858
[27]	validation_0-rmse:0.03858
[28]	validation_0-rmse:0.03859
[29]	validation_0-rmse:0.03859
[30]	validation_0-rmse:0.03859
[31]	validation_0-rmse:0.03859
[32]	validation_0-

# Save Model

# Model Test

In [42]:
base_path = 'ckpt/ml/ensemble'

In [46]:
model1_1 = XGBRegressor()
model1_1.load_model(base_path+'/xgb_ens1_0.model')

model1_2 = XGBRegressor()
model1_2.load_model(base_path+'/xgb_ens1_1.model')


model2_1 = XGBRegressor()
model2_1.load_model(base_path+'/xgb_ens2_0.model')

model2_2 = XGBRegressor()
model2_2.load_model(base_path+'/xgb_ens2_1.model')

In [57]:
mid_1 = model1_1.predict(test_x[:, wc1])
mid_2 = model1_2.predict(test_x[:, wc2])

mid = 0.5* mid_1 + 0.5*mid_2

In [58]:
pred_1 = model2_1.predict(mid[:, sc1])
pred_2 = model2_2.predict(mid[:, sc1])
pred = 0.5*pred_1+ 0.5*pred_2

# Get Accuracy

In [59]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def get_accuracy(y_hat, target):
    mae = mean_absolute_error(target, y_hat)
    mse = mean_squared_error(target, y_hat)
    rmse = np.sqrt(mse)
    
#     mape = torch.mean(torch.abs((target - y_hat) / target)) * 100

    acc = [round(mae, 4), round(mse, 4), round(rmse, 4)]
    return acc

In [60]:
get_accuracy(pred, test_y)

[0.0502, 0.004, 0.0629]

0.06194925823000602